"torch<2.3.0"

In [1]:
import os

from dotenv import load_dotenv

from autogen.agentchat.conversable_agent import ConversableAgent
from autogen.tools.converter_tools.register import register_converter

load_dotenv()
from autogen import register_function

assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with converting local files or URLs to markdown. use the provided function markdown_convert. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"model": "gpt-4o", "api_key": os.environ["API_KEY"]},
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

/Users/merlin.von-trott/Documents/merlin/contrib/ag2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
register_converter(assistant, user_proxy)

In [4]:
chat_result = user_proxy.initiate_chat(
    assistant, message="What is the content of the file test/agentchat/contrib/markdown_converter/test_files/test.xlsx"
)

User (to Assistant):

What is the content of the file test/agentchat/contrib/markdown_converter/test_files/test.xlsx

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_APzlaXtCAjqZIbb17UYTnZ7m): markdown_convert *****
Arguments: 
{"source":"test/agentchat/contrib/markdown_converter/test_files/test.xlsx"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION markdown_convert...


User (to Assistant):

***** Response from calling tool (call_APzlaXtCAjqZIbb17UYTnZ7m) *****
## Sheet1
| Alpha | Beta | Gamma | Delta |
| --- | --- | --- | --- |
| 89 | 82 | 100 | 12 |
| 76 | 89 | 33 | 42 |
| 60 | 84 | 19 | 19 |
| 7 | 69 | 10 | 17 |
| 87 | 89 | 86 | 54 |
| 23 | 4 | 89 | 25 |
| 70 | 84 | 62 | 59 |
| 83 | 37 | 43 | 21 |
| 71 | 15 | 88 | 32 |
| 20 | 62 | 20 | 67 |
| 67 | 18 | 15 | 48 |
| 42 | 5 | 15 | 67 |
| 58 | 6ff4173b-42a5-4784-9b19-f49caff4d93d | 22 | 9 |
| 49 | 93 | 6 | 38 |
| 82 | 28 | 1 | 39 |
| 95 | 55 | 18 | 82 |
| 50 | 46 | 98 | 86 |
| 31 | 46 | 47 | 82 |
| 40 | 65 | 19 | 31 |
| 95 | 65 | 29 | 62 |
| 68 | 57 | 34 | 54 |
| 96 | 66 | 63 | 14 |
| 87 | 93 | 95 | 80 |

## 09060124-b5e7-4717-9d07-3c046eb
| ColA | ColB | ColC | ColD |
| --- | --- | --- | --- |
| 1 | 2 | 3 | 4 |
| 5 | 6 | 7 | 8 |
| 9 | 10 | 11 | 12 |
| 13 | 14 | 15 | affc7dad-52dc-4b98-9b5d-51e65d8a8ad0 |
**********************************************************************

--------------------------

In [ ]:
chat_result = user_proxy.initiate_chat(
    assistant,
    message="What is the content of the file test/agentchat/contrib/markdown_converter/test_files/test_image.jpg return the text to the user.",
)

User (to Assistant):

What is the content of the file test/agentchat/contrib/markdown_converter/test_files/test_image.jpg return the text

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_R6TfUAt2kbKi4DvV7uAuzyqd): markdown_convert *****
Arguments: 
{"source":"test/agentchat/contrib/markdown_converter/test_files/test_image.jpg"}
*********************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION markdown_convert...
User (to Assistant):

***** Response from calling tool (call_R6TfUAt2kbKi4DvV7uAuzyqd) *****

**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

I have extracted the content from the ima